In [9]:
import pandas as pd
import zipfile
from os.path import exists

In [10]:
prep_dataset = None
if exists('merged_dataset.csv'):
    dataset = pd.read_csv('merged_dataset.csv')
    dataset = dataset.drop(columns=['Plot','Title', 'Unnamed: 0'])
    dataset.head()
    prep_dataset = dataset
elif exists('prepared.zip'):
    with zipfile.ZipFile('prepared.zip', 'r') as zip_ref:
        zip_ref.extractall('.')
        prep_dataset = pd.read_csv('prepared.csv')
prep_dataset.head()

,Unnamed: 0,Genre,Processed plot
0,0,"cult, horror, gothic, murder, atmospheric",note : thi synopsi is for the orgin italian re...
1,1,violence,"two thousand year ago , nhagruul the foul , a ..."
2,2,romantic,"matuschek ' s , a gift store in budapest , is ..."
3,3,"inspiring, romantic, stupid, feel-good","glenn holland , not a morn person by anyon ' s..."
4,4,"cruelty, murder, dramatic, cult, violence, atm...","in may 1980 , a cuban man name toni montana ( ..."
